In [ ]:
#!/usr/bin/env python3
import os
import time
import glob
import subprocess
import psutil

RESCTRL = "/sys/fs/resctrl"
MGROUP = "g1"
BENCH = "/home/cc/power/GPGPU/script/run_benchmark/npb_script/big/mg.sh"

TASKS = f"{RESCTRL}/mon_groups/{MGROUP}/tasks"
MONDATA = f"{RESCTRL}/mon_groups/{MGROUP}/mon_data"

def ensure_group():
    subprocess.run("sudo mount -t resctrl resctrl /sys/fs/resctrl || true",
                   shell=True, check=False)
    subprocess.run(f"sudo mkdir -p {RESCTRL}/mon_groups/{MGROUP}",
                   shell=True, check=False)

def attach_tid(tid):
    subprocess.run(f"echo {tid} | sudo tee {TASKS}",
                   shell=True, check=True, stdout=subprocess.DEVNULL)

def attach_all_threads(pid):
    tids = [p.split("/")[-1] for p in glob.glob(f"/proc/{pid}/task/*")]
    for tid in tids:
        attach_tid(tid)
    return set(tids)

def attach_threads_continuously(pid, seen, interval=0.2):
    # keep catching newly created worker threads
    try:
        while psutil.pid_exists(pid):
            tids = [p.split("/")[-1] for p in glob.glob(f"/proc/{pid}/task/*")]
            new = [t for t in tids if t not in seen]
            for tid in new:
                attach_tid(tid)
                seen.add(tid)
            time.sleep(interval)
    except KeyboardInterrupt:
        pass

def read_mbm_total_bytes():
    total = 0
    for entry in os.listdir(MONDATA):
        fpath = os.path.join(MONDATA, entry, "mbm_total_bytes")
        if os.path.isfile(fpath):
            with open(fpath) as fh:
                val = int(fh.read().strip())
                total += val
    return total

def main():
    ensure_group()

    # start MG
    proc = subprocess.Popen(BENCH)
    pid = proc.pid
    print(f"Started MG with PID {pid}")

    # initial attach
    time.sleep(0.2)
    seen = attach_all_threads(pid)
    print(f"Attached {len(seen)} threads initially")

    # start a lightweight background attach loop
    # you can inline this loop if you prefer a single thread
    last_attach_check = time.time()

    # MBM sampling
    prev = read_mbm_total_bytes()
    start = last = time.time()

    while proc.poll() is None:
        # keep attaching new threads every 200 ms
        if time.time() - last_attach_check >= 0.2:
            tids = [p.split("/")[-1] for p in glob.glob(f"/proc/{pid}/task/*")]
            new = [t for t in tids if t not in seen]
            for tid in new:
                attach_tid(tid)
                seen.add(tid)
            last_attach_check = time.time()

        # print MB per s every second
        time.sleep(1.0)
        cur = read_mbm_total_bytes()
        now = time.time()
        delta = max(cur - prev, 0)
        dt = max(now - last, 1e-6)
        mbps = delta / (1024 * 1024) / dt
        elapsed = now - start
        print(f"[{elapsed:6.1f}s] {mbps:8.1f} MB/s")
        prev, last = cur, now

    print("MG finished")

if __name__ == "__main__":
    main()


mount: /sys/fs/resctrl: resctrl already mounted on /sys/fs/resctrl.


Started MG with PID 273038


 NAS Parallel Benchmarks (NPB3.4-OMP) - MG Benchmark

 No input file. Using compiled defaults 
Attached 1 threads initially
 Size: 1024x1024x1024  (class U)
 Iterations:                     15
 Number of available threads:   160

[   1.0s]      0.0 MB/s
 Initialization time:           2.044 seconds

  iter   1
[   2.0s]      0.0 MB/s
[   3.0s]      0.0 MB/s
[   4.0s]      0.0 MB/s
  iter   5
[   5.0s]      0.0 MB/s
[   6.0s]      0.0 MB/s
[   7.0s]      0.0 MB/s
[   8.0s]      0.0 MB/s
  iter  10
[   9.0s]      0.0 MB/s
[  10.0s]      0.0 MB/s
[  11.0s]      0.0 MB/s
  iter  15
[  12.0s]      0.0 MB/s

 Benchmark completed 
 Problem size unknown
 NO VERIFICATION PERFORMED
 L2 Norm is  0.8412060065248E-06


 MG Benchmark Completed.
 Class           =                        U
 Size            =           1024x1024x1024
 Iterations      =                       15
 Time in seconds =                    11.04
 Total threads   =                      160
 Avail thr